# GraphStack

This notebook creates a simple python stack to create graph structures within the database(using postgresql and psycopg) and to manipulate them using networkx.

In [31]:
# import psycopg2

conn = psycopg2.connect(database="reddb", user="postgres", password="postgres", host="127.0.0.1", port="5433")
print("Database connection successful")

cursor = conn.cursor()
print("Got the cursor for this postgres connection")

cursor.execute('''CREATE TABLE COMPANY (
    id           INT PRIMARY KEY NOT NULL,
    Name         VARCHAR(20) NOT NULL,
    Designation  CHAR(3)     NOT NULL,
    DependId     INT         NOT NULL);''')
print("Table created successfully")

conn.commit()
     


Database connection successful
Got the cursor for this postgres connection


ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 39))



ProgrammingError: relation "company" already exists


In [50]:
# insert into the table, the table holds recursive relations, the root node will have dependId = 0

cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (1, 'Eng1', 'Eng', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (2, 'Eng2', 'Eng', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (3, 'Eng3', 'Eng', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (4, 'Eng4', 'Eng', 7)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (5, 'Eng5', 'Eng', 7)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (6, 'Pro1', 'PrM', 8)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (7, 'Pro2', 'PrM', 8)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DependId) VALUES (8, 'CTO1', 'CTO', 0)")

print("inserted into table successfully")

inserted into table successfully


In [35]:
# try to retrieve from the table

cursor.execute("SELECT id, name, Designation, DependId FROM COMPANY")
rows = cursor.fetchall()
for row in rows :
    print("Not Printing")
    print(row[0], row[1], row[2], row[3], "\n")


Not Printing
1 Eng1 Eng 6 

Not Printing
2 Eng2 Eng 6 

Not Printing
3 Eng3 Eng 6 

Not Printing
4 Eng4 Eng 7 

Not Printing
5 Eng5 Eng 7 

Not Printing
6 Pro1 PrM 8 

Not Printing
7 Pro2 PrM 8 

Not Printing
8 CTO1 CTO 0 



In [54]:
# perform recursive CTE on the table to get the graph structure starting from the root

cursor.execute('''WITH RECURSIVE traverse AS(
    SELECT id, Name from COMPANY
    WHERE id = 8
    UNION
    SELECT COMPANY.id, COMPANY.Name from COMPANY, traverse
    WHERE COMPANY.DependId = traverse.id)
    SELECT id, Name from traverse;
    ''')

recursive_rows = cursor.fetchall()
for row in recursive_rows :
    print("Printing")
    print(row[0], row[1], "\n")


Printing
8 CTO1 

Printing
6 Pro1 

Printing
7 Pro2 

Printing
1 Eng1 

Printing
2 Eng2 

Printing
3 Eng3 

Printing
4 Eng4 

Printing
5 Eng5 



In [56]:
# roll back the current transaction, due to previous error
# conn.rollback()

# conn.commit()

conn.close()


In [13]:
import networkx